In [1]:
import correct_lidar
import open3d as o3d
from pyntcloud import PyntCloud
# transform 
import numpy as np
import matplotlib.pyplot as plt
# from open3d import JVisualizer
import glob
import rasterio as rio



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# get all files in the data/l2/snow_off directory
files = glob.glob("data/l2/snow_on/*.lvx")
lidar = "l2"
for file in files:
    points = correct_lidar.load_first_file(lidar, file_search=file)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points.T)
    o3d.io.write_point_cloud(filename=f"./data/l2/snow_on/{lidar}_2022-10-23_01-00.pcd", pointcloud=pcd)

In [ ]:
lidars = ["l1", "l2", "l4", "l5", "l6"]
for lidar in lidars:
    points = correct_lidar.load_first_file(lidar, file_search=f"./data/snow_off/{lidar}_2022-10-23_*.lvx")
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points.T)
    o3d.io.write_point_cloud(filename=f"./data/snow_off/{lidar}_2022-10-23_01-00.pcd", pointcloud=pcd)
# o3d.visualization.draw_geometries([pcd])

# visualizer = JVisualizer()
# visualizer.add_geometry(pcd)
# visualizer.show()

In [15]:
points = correct_lidar.read_file(filename=f"./data/snow_off/{lidar}_2022-10-23_*.lvx")


array([63.987, 24.869, 37.463, 28.   ,  1.   , 16.   ])

In [28]:
points[:,:3].T

array([[64.042, 63.987, 63.831, ..., 10.543, 10.574, 10.632],
       [24.978, 24.869, 24.719, ..., -5.847, -5.831, -5.83 ],
       [37.461, 37.463, 37.402, ..., -0.958, -0.966, -0.975]])

In [29]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[:,:3])
o3d.io.write_point_cloud(filename=f"./data/snow_off/{lidar}_test.pcd", pointcloud=pcd)

True

In [54]:
import rioxarray as rxr

# open the raster
raster = rxr.open_rasterio("data/raster.tif", masked=True).squeeze()

# print the number of nans
print(f"Number of NaNs: {np.isnan(raster).sum()}")
# print the number of non-nans
print(f"Number of non-NaNs: {np.count_nonzero(~np.isnan(raster))}")
raster.plot.imshow(
    #increase point size
    cmap="viridis",
    size=10,
    aspect=1.5,
    robust=True,
)

Number of NaNs: <xarray.DataArray ()>
array(3426220)
Coordinates:
    band         int32 1
    spatial_ref  int32 0
Number of non-NaNs: 51980


In [ ]:
def correct_data(points, azimuth, xoffset, yoffset, zoffset, roll=0, elevation_adjustment=0,
                 global_adjustment=True, background_correction=None, correct_l5=False):
    """
    Apply corrections to raw lidar data for known lidar positions/orientations and to correct background light effects.
    Optionally do not reposition points into a common reference across lidars.
    Background light correction only applied if calibration targets are specified.
    """

    # first take out 30deg downward pointing mount for all lidars
    if elevation_adjustment==0:
        pr = rotate_data(v=points[:,:3].T, axis=[0, 0, 1], theta=np.deg2rad(-30.0), rotation=None)   # 31 deg
    else:
        pr = rotate_data(v=points[:,:3].T, axis=[0, 0, 1], theta=np.deg2rad(-30.0+elevation_adjustment), rotation=None)   # 31 deg

    # Then account for mounting on a vertical mount
    if roll==0:
        pr = rotate_data(pr, axis=[1,0,0],theta=np.pi/2)
    else:
        pr = rotate_data(pr, axis=[1,0,0],theta=np.pi/2 + np.deg2rad(roll))

    if correct_l5:
        pr = update_l5(pr)

    if background_correction is not None:
        pr = correct_points_background(pr, background_correction)


    if global_adjustment:
        # Then rotate to a common map orientation
        pr = rotate_data(pr, axis=[0,0,1],theta=np.deg2rad(azimuth))

        # shift location based on lidar mount points
        pr[0,:] += (xoffset - center_point[0])
        pr[1,:] += (yoffset - center_point[1])
        pr[2,:] += zoffset

        # Then rotate to a North map orientation
        pr = rotate_data(pr, axis=[0,0,1],theta=np.deg2rad(global_rotation))


    return pr